In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.python.client import device_lib
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import resample, shuffle


2024-10-02 13:14:19.732183: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 13:14:20.407449: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 13:14:20.484683: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-02 13:14:21.173323: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-02 13:14:24.156821: W tensorflow/compiler/tf2

In [2]:
# Check if TensorFlow is using a GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("TensorFlow is using the following GPU(s):")
    for gpu in gpus:
        print(gpu)
else:
    print("No GPU detected for TensorFlow.")


TensorFlow is using the following GPU(s):
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [3]:
import tensorflow as tf
print(tf.__version__)


2.17.0


In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [5]:
# Paths
data_dir = 'data/training'  # Folder with .tif images
csv_file = 'data/training.csv'  # CSV file with image_id and is_homogeneous


In [6]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 16

In [7]:
df = pd.read_csv(csv_file)

In [8]:
# Strip any leading/trailing spaces from column names
df.columns = df.columns.str.strip()

# Now you can access 'image_id' without the extra space
image_id_values = df['image_id'].values
print(image_id_values)


[279 277 275 273 271 269 267 265 263 261 259 257 255 253 251 249 247 245
 243 241 239 237 235 233 231 229 227 225 223 221 219 217 215 213 211 209
 207 205 203 201 199 197 195 193 191 189 187 185 183 181 179 177 175 173
 171 169 167 165 163 161 159 157 155 153 151 149 147 145 143 141 139 137
 135 133 131 129 127 125 123 121 119 117 115 113 111 109 107 105 103 101
  99  97  95  93  91  89  87  85  83  81  79  77  75  73  71  69  67  65
  63  61  59  57  55  53  51  49  47  45  43  41  39  37  35  33  29  27
  25  23  21  19  17  15  13  11   9   7   5   3   1]


In [9]:
def load_and_preprocess_image(image_path):
    # Load image with PIL and convert to an array
    img = load_img(image_path, target_size=IMG_SIZE)
    img_array = img_to_array(img)
    # Normalize image pixel values (0-255 -> 0-1)
    img_array = img_array / 255.0
    return img_array

In [10]:
print(df.columns)



Index(['image_id', 'is_homogenous'], dtype='object')


In [11]:
# 3. Create lists of image paths and labels
# Assuming image_id values need to be 3 digits with leading zeros
image_paths = [os.path.join(data_dir, f"{str(image_id).zfill(3)}.tif") for image_id in df['image_id']]
labels = df['is_homogenous'].values

In [12]:
# 4. Load images and preprocess them
images = np.array([load_and_preprocess_image(image_path) for image_path in image_paths])



In [13]:
# 5. Split the data into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)


In [14]:
# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

In [15]:
class_weights_dict = dict(enumerate(class_weights))
print(f"Class weights: {class_weights_dict}")

Class weights: {0: 0.6098901098901099, 1: 2.775}


In [16]:
class_weights_dict = {0: 1.0, 1: 2.0}


In [17]:
# 6. Create data generators for augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

In [18]:
val_datagen = ImageDataGenerator()


In [19]:
# 7. Create the data generators
train_generator = train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)
val_generator = val_datagen.flow(X_val, y_val, batch_size=BATCH_SIZE)


In [20]:
# 8. Load the pre-trained VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


2024-10-02 13:14:41.816707: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38137 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:01:00.0, compute capability: 8.0


In [21]:
for layer in base_model.layers:
    layer.trainable = False

In [22]:
# 10. Create the model by adding custom layers on top of the pre-trained base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Add dropout for regularization
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

In [23]:
import tensorflow_addons as tfa  # same as regular TensorFlow Addons

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss=tfa.losses.SigmoidFocalCrossEntropy(),
    metrics=['accuracy']
)

/home/cogu/.conda/envs/cellclass/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/cogu/.conda/envs/cellclass/lib/python3.9/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.13.0 and strictly below 2.16.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.17.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file a

In [ ]:
history = model.fit(
    train_generator,
    epochs=20,  # Adjust the number of epochs as needed
    validation_data=val_generator,
    class_weight=class_weights_dict
)

Epoch 1/20


/home/cogu/.conda/envs/cellclass/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1727867686.558744   10951 service.cc:146] XLA service 0x7f9b7000c120 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727867686.558785   10951 service.cc:154]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-10-02 13:14:46.626644: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-02 13:14:52.643003: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-10

2/7 ━━━━━━━━━━━━━━━━━━━━ 23s 5s/step - accuracy: 0.5938 - loss: 2.0708  

In [ ]:
# 13. Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_acc*100:.2f}%")

In [ ]:
model.save('vgg16_homogeneous_classification.h5')


In [ ]:
from tensorflow.keras.models import load_model


In [ ]:
model1 = load_model('vgg16_homogeneous_classification.h5')  # Path to your saved model


In [ ]:
val_loss, val_accuracy = model.evaluate(X_val, y_val)


In [ ]:
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_acc * 100:.2f}%')

In [ ]:
predictions = model.predict(X_val)
predicted_labels = (predictions >= 0.5).astype(int)  # Threshold at 0.5 to get binary labels


In [ ]:
from sklearn.metrics import classification_report


In [ ]:
print(classification_report(y_val, predicted_labels, target_names=['Heterogeneous', 'Homogeneous']))


In [ ]:

# Assuming y_val contains the true labels and predicted_labels contains the predicted labels

# Step 1: Calculate n_0 and n_1
n_0 = np.sum(y_val == 0)  # Number of true heterogeneous cells
n_1 = np.sum(y_val == 1)  # Number of true homogeneous cells

# Step 2: Calculate a_0 and a_1
a_0 = np.sum((y_val == 0) & (predicted_labels == 0))  # Correctly predicted as heterogeneous
a_1 = np.sum((y_val == 1) & (predicted_labels == 1))  # Correctly predicted as homogeneous

# Step 3: Calculate the score
if n_0 == 0 or n_1 == 0:
    score = 0  # Handle edge cases where there are no samples of a class
else:
    score = (a_0 * a_1) / (n_0 * n_1)

print(f'Score: {score}')
